In [1]:
import pandas as pd
import psycopg2

import os
#os.getcwd()
conn_str = "dbname=postgres user=postgres password=asdf123"


In [2]:

def get_all_table_names(conn_str):
    conn = psycopg2.connect(conn_str)
    sql = """
    SELECT table_name FROM INFORMATION_SCHEMA.TABLES
    WHERE table_schema = N'old_data'
    """


    questionnaire_columns = list(pd.read_sql_query(sql, conn).table_name)
    conn.close()
    return questionnaire_columns

## get column names

In [3]:
def get_column_names(questionnaire_name, conn_str):
    conn = psycopg2.connect(conn_str)
    sql = """
    SELECT *
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = N'{0}'
    """.format(questionnaire_name)

    questionnaire_columns = list(pd.read_sql_query(sql, conn).column_name)
    conn.close()
    return questionnaire_columns

In [4]:
def prepare_data(questionnaire_name, questionnaire_columns, old_data_df):
    questionnaire_data = old_data_df[questionnaire_columns]

    file_name = "old_{0}_data.csv".format(questionnaire_name)
    path = r"..\Data\postgres_db\old_data\{0}".format(file_name)
    questionnaire_data.to_csv(path, index=False)
    
    # Change the file permissions
    os.chmod(path, 0o777)
    
    return file_name


In [5]:
def upload_data(questionnaire_name, file_name, conn_str):
    upload_data = """
    COPY old_data.{0}
    FROM '/postgres_db/old_data/{1}'
    DELIMITER ','
    CSV HEADER;

    """.format(questionnaire_name, file_name)


    conn = psycopg2.connect(conn_str)
    # Open a cursor to perform database operations
    cur = conn.cursor()
    # Execute a command: create datacamp_courses table
    cur.execute(upload_data)
    
    
    # Make the changes to the database persistent
    conn.commit()

    # Close cursor and communication with the database
    cur.close()
    conn.close()

In [6]:
old_data_df = pd.read_csv(r"..\Data\Transferring_Data_to_PostgreSQL\old_data.csv")
table_names = get_all_table_names(conn_str)


C:\Users\nogur\anaconda3\envs\DeppClinic\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1593) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Prepare the needed table

In [7]:

for questionnaire_name in table_names:

    questionnaire_columns = get_column_names(questionnaire_name, conn_str)
    file_name = prepare_data(questionnaire_name, questionnaire_columns, old_data_df)
    upload_data(questionnaire_name, file_name, conn_str)
    print(questionnaire_name)


UndefinedFile: could not open file "/postgres_db/old_data/old_c_ssrs_intake_data.csv" for reading: No such file or directory
HINT:  COPY FROM instructs the PostgreSQL server process to read a file. You may want a client-side facility such as psql's \copy.


In [ ]:
table_names